# Operando con datos en Pandas

Uno de los puntos fuertes de NumPy es que nos permite realizar operaciones rápidas elemento a punto, tanto con aritmética básica (suma, resta, multiplicación, etc.) como con operaciones más complicadas (funciones trigonométricas, funciones exponenciales y logarítmicas, etc.) .
Pandas hereda gran parte de esta funcionalidad de NumPy, y las ufuncs introducidas en [Computación en matrices NumPy: funciones universales] (02.03-Computación-en-arrays-ufuncs.ipynb) son clave para esto.

Sin embargo, Pandas incluye un par de cambios útiles: para operaciones unarias como negación y funciones trigonométricas, estas ufuncs *preservarán las etiquetas de índice y columna* en la salida, y para operaciones binarias como suma y multiplicación, Pandas automáticamente *alineará índices* al pasar los objetos al ufunc.
Esto significa que mantener el contexto de los datos y combinar datos de diferentes fuentes (ambas tareas potencialmente propensas a errores con matrices NumPy sin procesar) se vuelven esencialmente infalibles con Pandas.
Además veremos que existen operaciones bien definidas entre estructuras unidimensionales `Series` y estructuras bidimensionales `DataFrame`.

## Ufuncs: Preservación del índice

Debido a que Pandas está diseñado para funcionar con NumPy, cualquier ufunc de NumPy funcionará en los objetos `Series` y `DataFrame` de Pandas.
Comencemos definiendo una `Serie` y un `DataFrame` simples para demostrar esto:

In [1]:
import pandas as pd
import numpy as np

In [2]:
rng = np.random.default_rng(42)
ser = pd.Series(rng.integers(0, 10, 4))
ser

0    0
1    7
2    6
3    4
dtype: int64

In [3]:
df = pd.DataFrame(rng.integers(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,4,8,0,6
1,2,0,5,9
2,7,7,7,7


Si aplicamos una ufunc NumPy en cualquiera de estos objetos, el resultado será otro objeto Pandas *con los índices conservados:*

In [4]:
np.exp(ser)

0       1.000000
1    1096.633158
2     403.428793
3      54.598150
dtype: float64

Esto también es válido para secuencias de operaciones más complicadas:

In [5]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,1.224647e-16,-2.449294e-16,0.000000,-1.000000
1,1.000000e+00,0.000000e+00,-0.707107,0.707107
2,-7.071068e-01,-7.071068e-01,-0.707107,-0.707107


Cualquiera de las ufuncs analizadas en [Cálculo en matrices NumPy: funciones universales] (02.03-Computación-en-arrays-ufuncs.ipynb) se puede utilizar de manera similar.

## Ufuncs: Alineación del índice

Para operaciones binarias en dos objetos `Series` o `DataFrame`, Pandas alineará los índices en el proceso de realizar la operación.
Esto es muy conveniente cuando se trabaja con datos incompletos, como veremos en algunos de los ejemplos siguientes.

### Alineación de índice en serie

Como ejemplo, supongamos que estamos combinando dos fuentes de datos diferentes y deseamos encontrar solo los tres principales estados de EE. UU. por *área* y los tres principales estados de EE. UU. por *población*:

In [6]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 39538223, 'Texas': 29145505,
                        'Florida': 21538187}, name='population')

Veamos qué sucede cuando los dividimos para calcular la densidad de población:

In [7]:
population / area

Alaska              NaN
California    93.257784
Florida             NaN
Texas         41.896072
dtype: float64

La matriz resultante contiene la *unión* de índices de las dos matrices de entrada, que podrían determinarse directamente a partir de estos índices:

In [8]:
area.index.union(population.index)

Index(['Alaska', 'California', 'Florida', 'Texas'], dtype='object')

Cualquier elemento para el cual uno u otro no tenga una entrada se marca con `NaN`, o "No es un número", que es como Pandas marca los datos faltantes (consulte más información sobre los datos faltantes en [Manejo de datos faltantes](03.04- Valores-perdidos.ipynb)).
Esta coincidencia de índice se implementa de esta manera para cualquiera de las expresiones aritméticas integradas de Python; cualquier valor faltante está marcado con "NaN":

In [9]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

Si usar valores `NaN` no es el comportamiento deseado, el valor de relleno se puede modificar usando métodos de objeto apropiados en lugar de los operadores.
Por ejemplo, llamar a ``A.add(B)`` es equivalente a llamar a ``A + B``, pero permite la especificación explícita opcional del valor de relleno para cualquier elemento en ``A`` o ``B`` que podría faltar:

In [10]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### Alineación de índice en marcos de datos

Un tipo similar de alineación tiene lugar para *ambas* columnas e índices al realizar operaciones en objetos `DataFrame`:

In [11]:
A = pd.DataFrame(rng.integers(0, 20, (2, 2)),
                 columns=['a', 'b'])
A

,a,b
0,10,2
1,16,9


In [12]:
B = pd.DataFrame(rng.integers(0, 10, (3, 3)),
                 columns=['b', 'a', 'c'])
B

,b,a,c
0,5,3,1
1,9,7,6
2,4,8,5


In [13]:
A + B

,a,b,c
0,13.0,7.0,NaN
1,23.0,18.0,NaN
2,NaN,NaN,NaN


Observe que los índices están alineados correctamente independientemente de su orden en los dos objetos y los índices del resultado están ordenados.
Como fue el caso con `Series`, podemos usar los métodos aritméticos del objeto asociado y pasar cualquier `fill_value` que deseemos para usar en lugar de las entradas faltantes.
Aquí completaremos con la media de todos los valores en "A":

In [14]:
A.add(B, fill_value=A.values.mean())

,a,b,c
0,13.00,7.00,10.25
1,23.00,18.00,15.25
2,17.25,13.25,14.25


La siguiente tabla enumera los operadores de Python y sus métodos de objeto Pandas equivalentes:

| Operador Python | Método(s) de pandas |
|-----------------|------------------------------- --|
| `+` | `agregar` |
| `-` | `sub`, `resta` |
| `*` | `mul`, `multiplicar` |
| `/` | `truediv`, `div`, `dividir` |
| `//` | `pisodiv` |
| `%` | `mod` |
| `**` | `poder` |

## Ufuncs: Operaciones entre DataFrames y Series

Al realizar operaciones entre un `DataFrame` y una `Series`, la alineación del índice y la columna se mantiene de manera similar, y el resultado es similar a las operaciones entre una matriz NumPy bidimensional y unidimensional.
Considere una operación común, donde encontramos la diferencia entre una matriz bidimensional y una de sus filas:

In [15]:
A = rng.integers(10, size=(3, 4))
A

array([[4, 4, 2, 0],
       [5, 8, 0, 8],
       [8, 2, 6, 1]])

In [16]:
A - A[0]

array([[ 0,  0,  0,  0],
       [ 1,  4, -2,  8],
       [ 4, -2,  4,  1]])

De acuerdo con las reglas de transmisión de NumPy (consulte [Computación en matrices: transmisión] (02.05-Computation-on-arrays-broadcasting.ipynb)), la resta entre una matriz bidimensional y una de sus filas se aplica fila por fila.

En Pandas, la convención opera de manera similar en filas de forma predeterminada:

In [17]:
df = pd.DataFrame(A, columns=['Q', 'R', 'S', 'T'])
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,1,4,-2,8
2,4,-2,4,1


Si, en cambio, desea operar en columnas, puede utilizar los métodos de objeto mencionados anteriormente, mientras especifica la palabra clave `axis`:

In [18]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,0,0,-2,-4
1,-3,0,-8,0
2,6,0,4,-1


Tenga en cuenta que estas operaciones `DataFrame`/`Series`, como las operaciones analizadas anteriormente, alinearán automáticamente los índices entre los dos elementos:

In [19]:
halfrow = df.iloc[0, ::2]
halfrow

Q    4
S    2
Name: 0, dtype: int64

In [20]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,1.0,NaN,-2.0,NaN
2,4.0,NaN,4.0,NaN


Esta preservación y alineación de índices y columnas significa que las operaciones con datos en Pandas siempre mantendrán el contexto de los datos, lo que evita los errores comunes que pueden surgir al trabajar con datos heterogéneos y/o desalineados en matrices NumPy sin procesar.